# Modelo da MAV em forma de grafo para simulação com Nanorobôs

A MAV é representada em foram de grafo e um enxame de nanorobôs
é injetado na circulação para embolizar os vasos malformados.

In [19]:
# SPDX-License-Identifier: GPL-3.0-or-later
""" Modelo da MAV em forma de grafo para simulação com Nanorobôs.

.. codeauthor:: Carlo Oliveira <carlo@ufrj.br>

Changelog
---------

.. versionchanged::    22.05

    Add scatter plot nanite x node

.. versionchanged::    21.07.b

    Reformat Jupyter with more cells

.. versionadded::    21.07.a

    Start data collection and tentative plotting

.. versionadded::    21.06.e

    Refactor out Embolize class

.. versionadded::    21.07.d

    Saving vases

.. versionadded::    21.06.c

    Grafo nanite e simulação da embolização

.. versionadded::    21.06.b

    Delimita mav e inicia criação de aresta

.. versionadded::    21.06.a

    Grafo nanite
"""
__version__ = "22.05"

## Caracterização de um Nó do Sistema Circulatório

Este Objeto caracteriza um nó entroncamento de vasos a ser usado no grafo do sistema circulatório.
Ele incorpora a capacidade de participar da simulação lidando com a passage de nanites por ele.
Ao passar por ele o nanite terá que obdecer a capacidade de fluxo deste nó.
O nanite receberá a características biofísicoquímicas do nó para ajustar o seu comportamento.

In [20]:
class Node:
    def __init__(self, env, syst, nome, ndepth, nmav):
        """ Nó construído para suportar o processo de simulação.

        :param env: Ambiente de simulação
        :param syst: Sistema circulatório que contem este nó.
        :param nome: Nome deste nó.
        :param ndepth: Profundidade corrente no grafo arterial.
        :param nmav: determina o nível de marcadores de mav neste nó.
        """
        self.env, self.nome, self.depth, self.mav = env, nome, ndepth, nmav
        self.dest, self.cross, self.embolized = [], 0, 0
        self.flow = simpy.Resource(env, ndepth)
        """Capacidade de vazão deste nó, indo de grosso calibre a capilar"""
        syst.setdefault(ndepth, []).append(self)

    def add_edge(self, ndest):
        """ Adiciona uma vaso de saída neste entroncamento.

        :param ndest: Nó destino
        :return: None
        """
        self.dest.append(ndest)

    def set_edges(self, ndest):
        """ Substitui a coleção de vasos de saída neste entroncamento.

        :param ndest: lista de nós destino
        :return: None
        """
        self.dest = ndest

    def embolize(self):
        """ Emboliza este nó.

        :return: None
        """
        self.embolized += 1

    def cross_node(self, nanite):
        """ Atravessa um nanite por este nó.
        
        Ao passar pelo nó, o nanite terá que obedecer a capacidade de fluxo aqui presente.
        O nanite receberá as características biofisicoquímicas do nó para ajustar o seu comportamento.

        :param nanite: O nanite transeunte.
        :return: None
        """
        self.cross += 1
        # print("cross_node", nanite.name, self.ori, self.cross)
        yield self.env.process(nanite.cross_node(self.flow, self, self.dest, self.mav, self.depth))
        yield self.env.timeout(NODE_TRAVERSE)


## Caracterização do sistema circulatório e da MAV em forma de grafo

In [21]:
import simpy
import random
import statistics
import networkx as nx  # importing networkx package
import matplotlib.pyplot as plt  # importing matplotlib package and pyplot is for displaying the graph on canvas
import seaborn as sns

NODE_TRAVERSE = 0.01
MAV_THRESHOLD = 5
NODEINFO = "node_info"
DEST = "destinations"
VASE_FILE = "artery_only.gpycle"

class Mav(nx.Graph):
    """Gráfico MAV simulando a estrutura de vasos sanguíneos no cérebro.

    """
    VASE_LEVEL = {}
    """Dicionário contendo todos os vasos ordenados pela distância do entroncamento"""
    def __init__(self, entry=8, mav_depth=6, graph_depth=7):
        """ Gráfico MAV simulando a estrutura de vasos sanguíneos no cérebro.

        :param entry: capacidade de vazão do injetor de nanites.
        :param mav_depth: profundidade na rede onde se inicia uma malformação.
        :param graph_depth: profundidade da rede de vasos sanguíneos.
        """
        self.env = None #env
        self.entry = entry
        super().__init__()
        self.node_name = 1
        """Inicia a nomeação dos nós com o número 1"""
        self.graph_depth = graph_depth
        self.mav_origin = False
        """Marcador do início da MAV no grafo."""
        self.mav_depth = mav_depth
        self.markers = "blue green yellow red yellow green".split()
        """Nível de concentração dos marcadores de MAV."""
        self.entry_node = None
        """Vaso onde os nanites foram injetados"""

    def do_edge(self, node, depth, mav=0):
        """ Criação de uma aresta representado um vaso sanguíneo

        :param node: O nó, que representa um entroncamento de vasos.
        :param depth: profundidade corrente, decrementa ao avançar de nível.
        :param mav: determina o nível de marcadores de mav neste nó
        :return: None
        """
        self.node_name += 1
        mav_depth = self.mav_depth
        name = self.node_name
        self.add_an_edge(node, name, depth=depth, mav=mav)
        self.nodes[name]['color'] = self.markers[mav]
        depth -= 1
        if (not self.mav_origin) and (depth < mav_depth):
            self.mav_origin = True
            mav = 1
        elif mav and (depth < mav_depth):
            mav += 1
        [self.do_edge(name, depth, mav=mav) for _ in range(0, random.randint(2, 3)) if depth]

    def embolized(self, threshold=1):
        """ Lista de nós embolizados.
        
        :return: Lista de nós embolizados
        """
        return [self.nodes[nd][NODEINFO].embolized
                                      for nd in self.nodes if self.nodes[nd][NODEINFO].embolized >= threshold]


    def redo_edge(self):
        """ Criação de um objeto Node para cada aresta representado um vaso sanguíneo com nó de processamento

        :return: None
        """
        env, sist = self.env, self.VASE_LEVEL


        for node_name in self.nodes: 
            node_depth, node_mav = self.nodes[node_name][NODEINFO]
            processing_node = Node(env, sist, node_name, node_depth, node_mav)
            self.nodes[node_name][NODEINFO] = processing_node
        for node_name in self.nodes: 
            self.nodes[node_name][NODEINFO].set_edges(
                [self.nodes[adjacent][NODEINFO] for adjacent in self.nodes[node_name][DEST]
                 if adjacent in self.nodes]) 

    def add_an_edge(self, ori, dest, depth, mav, **kwargs):
        """Adiciona uma aresta no NetworkX e cria uma classe Node para representar o destino.

        :param ori: Nó de origem.
        :param dest: nò de destino.
        :param depth: Profundidade corrente no grafo arterial.
        :param mav: determina o nível de marcadores de mav neste nó.
        :param kwargs: Outros parâmetros auxiliares para ajudar na plotagem.
        :return: None
        """
        super().add_edge(ori, dest, **kwargs)
        """Usa o add edge herdado do NetworkX"""
        node, ori_node = self.nodes[dest], self.nodes[ori]
        # return
        ori_node.setdefault(DEST,[]).append(dest)
        if NODEINFO not in ori_node:
            ori_node[NODEINFO] = (depth, mav)
        if NODEINFO not in node:
            node[NODEINFO] = (depth, mav)
            node.setdefault(DEST,[])


        # self.nodes[ori]["node"].add_edge(self.nodes[dest]["node"])

    def build(self, filename=VASE_FILE):
        """Constrói o grafo de maneira aleatória

        :return: None
        """
        self.do_edge(self.node_name, self.graph_depth)
        self.nodes[1]['color'] = "blue"
        nx.write_gpickle(self, filename)


    def load(self, env):
        """Recupera um grafo

        :return: None
        """
        self.env = env
        self.redo_edge()
        self.entry_node = self.nodes[1][NODEINFO]

    def plot(self):
        values = [self.nodes[node]["color"] for node in self.nodes()]

        nx.draw(self, with_labels=True, node_color=values)
        #  #draws the networkx graph containing nodes which are declared till before
        plt.show()  # displays the networkx graph on matplotlib canvas


### Estatísticas do Sistema Circulatório

In [26]:

def show_mav(build_mav=False, env=None, entry=8):
        # import seaborn as sns
        entry_point = simpy.Resource(env, entry)
        """capacidade de vazão da agulha injetora"""
        mav = Mav(entry=entry_point)
        # random.seed(43)
        mav.build() if build_mav else None
        mav = nx.read_gpickle(VASE_FILE)
        """representação da má formação"""
        mav.load(env=env)
        nodes = [(node, mav.nodes[node][NODEINFO].depth,
                  mav.nodes[node][NODEINFO].mav, len(mav.nodes[node][NODEINFO].dest)) for node in mav.nodes]
        nodes = zip(*nodes)
        campos ="nome depth mav dest".split()
        from pandas import DataFrame as DtF
        df = DtF({campo: valores for campo, valores in zip(campos, nodes)})
        sns.displot(data=df, hue="depth", x="mav", alpha=0.9, height=2, col="dest")
        # sns.scatterplot(data=df, x="depth", y="mav") #, hue="dest")
        return df

show_mav()

<IPython.core.display.Javascript object>

,nome,depth,mav,dest
0,1,7,0,1
1,2,7,0,2
2,3,6,0,2
3,4,5,1,2
4,5,4,2,3
...,...,...,...,...
272,273,1,0,0
273,274,1,0,0
274,275,2,0,2
275,276,1,0,0


## Caracterização do Nanorobô

In [23]:
        
class Nanite:
    def __init__(self, name, log, env, depth):
        """ O nanochip de embolização. Atualmente tem uma programação aleatória

        :param name: Identificador do chip.
        :param log: Método ao qual se deve reportar os eventos no trajeto.
        :param env: Ambiente de simulaçao.
        :param depth: Profundidade programada de embolização.
        """
        self.name, self.log, self.depth, self.env = name, log, depth, env
        self.mav, self.trs = 0, -1
        """Concentração de marcadores onde embolizou, nível de profundidade quando embolizou"""

    def cross_node(self, flow, current_node, destinations, mav, depth):
        """Atravessa um entrocamento de vasos, decidindo a ação a tomar.

        :param flow: capacidade de fluxo onde o nanite se encontra.
        :param current_node: Nó que está atravessando no momento.
        :param destinations: Listra dos possíveis destinos neste entroncamento.
        :param mav: Determina o nível de marcadores de mav neste nó.
        :param depth: Profundidade do nó que está atravessando no momento.
        :return:
        """
        if not self.depth:
            return
        with flow.request() as request:
            destination = 0
            yield request
            self.trs = depth
            if (depth <= self.depth) and mav:
                """estando na MAV e profundidade programada, para e emboliza"""
                self.mav = mav
                current_node.embolize()
                self.depth = 0
            elif destinations:
                destination = random.choice(destinations)
                yield self.env.process(destination.cross_node(self))
            self.log(self.name, current_node.nome, current_node.depth, current_node.mav,
                     destination.nome if destination else 0)


## Processo de Embolização

In [24]:
class Embolization:
    VASE_EVENT = {}
    """Dicionário de eventos da simulação indexado por nanaites"""
    def __init__(self, env, entry=8, build_mav=False):
        """ Controlador do processo de embolização.

        :param env: Ambiente de simulação.
        :param entry: capacidade de vazão do injetor de nanites.
        """
        self.env = env
        """Ambiente de simulação"""
        self.swarm = []
        """Armazena todos os nanites injetados no sangue"""
        self.entry_point = simpy.Resource(env, entry)
        """capacidade de vazão da agulha injetora"""
        mav = Mav(entry=self.entry_point)
        # random.seed(43)
        mav.build() if build_mav else None
        self.mav = mav = nx.read_gpickle(VASE_FILE)
        """representação da má formação"""
        mav.load(env=env)
        self.wait_times = []
        """tempos de percurso dos nanites"""

    def compute_event(self, nanite, node, level, mav, destination):
        """Computa eventos da travessia de um nanite por dentro da corrente sanguínea.
        
        Este método é injetado em cada nanite para que ele possa reportar os eventos do trajeto.
        
        :param nanite: Identificador do nanorobô que gerou o evento
        :param node: Identificador do nó circulatório onde o evento aconteceu.
        :param level: Nivel na árvore do nó circulatório onde o evento aconteceu.
        :param mav: Nível de marcadores de mav onde o evento aconteceu.
        :param destination: Identificador do nó circulatório para onde o nanorobô se dirige.
        """
        self.VASE_EVENT.setdefault(nanite, []).append(
            ((1000*self.env.now)//10, node, level, mav, destination))

    def go_to_mav(self, nanite):
        """Inicia a travessia de um nanite por dentro da corrente sanguínea.
        
        :param nanite: Identificador do nanorobô que gerou o evento
        """
        arrival_time = self.env.now

        with self.entry_point.request() as request:
            yield request
            # nanite_chip = Nanite(nanite, self.env, self.mav_depth)
            nanite_chip = Nanite(nanite, self.compute_event, self.env, MAV_THRESHOLD - random.randint(0, 5))
            self.swarm.append(nanite_chip)
            yield self.env.process(self.mav.entry_node.cross_node(nanite_chip))

        self.wait_times.append(self.env.now - arrival_time)
        """Armazena o intervalo de tempo atual na lista de intervalos"""

    def run_embolization(self, initial_nanites=3, nanite_interval=0.05):
        """ Executa o processo de embolização, injetando diversos nanites.

        :param initial_nanites:  Frente inicial de nanites injetados.
        :param nanite_interval: Intervalo para a injeção de um nanite
        :return: None
        """
        for nanite in range(initial_nanites):
            self.env.process(self.go_to_mav(nanite))

        nanite = initial_nanites

        while True:
           for nani in range(initial_nanites):
                yield self.env.timeout(nanite_interval)  # Wait a bit before generating a new nanite
                nanite_id = nanite + nani
                self.env.process(self.go_to_mav(nanite_id))
           nanite += initial_nanites

            #self.env.process(self.go_to_mav(nanite))

    def get_average_wait_time(self):
        average_wait = statistics.mean(self.wait_times)
        minutes, frac_minutes = divmod(average_wait, 1)
        seconds = frac_minutes * 60
        return round(minutes), round(seconds)

    def count_embolizations(self):
        """ Retorna valores sobre a embolização

        :return: Número de Manites que embolizaram, número de nós embolizados.
        """
        # print(statistics.mean(nano.trs for nano in self.swarm))
        # print(self.entry_node.cross, [[k.ori for k in self.nodes[j]["node"].dest] for j in range(2, 6)])
        # print(self.entry_node.dest)
        return sum(1 for nano in self.swarm if nano.mav), len(self.swarm), sum(
            1 for _ in self.mav.embolized()
                   )

    def count_mav_rates(self):
        """Retorna estatísticas sobre a MAV embolizada

        :return: Nós embolizados, média de nanites embolizando o mesmo nó.
        """
        return sum(1 for _ in self.mav.embolized(0)
                   ), statistics.mean(self.mav.embolized())



    def main(self, simulation_time=400):
        env = self.env
        # mav.plot() #descomente esta linha para plotar o grafo
        # [print(level, node.nome) for level, level_list in Mav.VASE_LEVEL.items() for node in level_list]
        # [print(level, [adj.nome for adj in node.dest]) for level, level_list in Mav.VASE_LEVEL.items() for node in level_list]
        # print (mav.adj)

        # Run the simulation
        env.process(self.run_embolization())
        env.run(until=simulation_time)

        # View the results
        mins, secs = self.get_average_wait_time()
        success, total, embolized = self.count_embolizations()
        mav_nodes, avg_embol = self.count_mav_rates()
        print(
          "Running simulation...",
          f"\nThe average wait time is {mins} minutes and {secs} seconds.",
          f"\nThe count of embolizing nanites is {success} over {total} total, {100*success/total: .2f}% efficient.",
          f"\nThe count of embolized vazes is {embolized} against exisiting {mav_nodes}  mav nodes.",
          f"\nThe process efficacy is {100*embolized/mav_nodes: .2f}% and over_embolization {avg_embol: .2f} average.",
        )
        event = {}
        for nanite, events in self.VASE_EVENT.items():
            [event.setdefault(eve[0], []).append(nanite) for eve in events]
        #print([len(nans) for nans in event.values()])
        #[print(f"nanite {nanite}, events {events}") for nanite, events in self.VASE_EVENT.items()]


## Relatório dos Resultados

In [25]:

class ReportResults:
    """ Gera relatórios do processo de embolização.
    """
    def __init__(self, event):
        """ Gera relatórios do processo de embolização.
        
        :param event:  Dicionário de eventos recebido do processo de embolização.
        """
        self.VASE_EVENT = event

    def splot(self):
        %matplotlib notebook
        # from mpl_toolkits.mplot3d import Axes3D
        # import matplotlib.pyplot as plt
        # import numpy as np
        #
        # Assuming you have "2D" dataset like the following that you need
        # to plot.
        #
        event = {}
        for nanite, events in self.VASE_EVENT.items():
            [event.setdefault(eve[0], []).append(nanite) for eve in events]
        evens = [(nanite, node, time//10, level, mav) for nanite, events in self.VASE_EVENT.items()
        for time, node, level, mav, *_ in events]
        # box_time = max([ev[2] for ev in evens])
        nanite, node, time, level, mav = zip(*evens)
        level = [8*(8-lev) for lev in level]
        nanite = [300-lev for lev in nanite]
        fig = plt.figure()
        fig.set_size_inches(12, 12)
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(xs = nanite, ys = node, zs = time, c=mav, s=level)
        ax.set_title("Nanite progression along nodes")
        ax.set_xlabel("nanite (id)")
        ax.set_ylabel("node (id)")
        ax.set_zlabel("Time (ms)")
        ax.set_zlim(0,100)
        plt.show()
    
    def hplot(self, data_array=None):
        # from mpl_toolkits.mplot3d import Axes3D
        # import matplotlib.pyplot as plt
        import numpy as np
        event = {}
        for nanite, events in self.VASE_EVENT.items():
            [event.setdefault(eve[0], []).append(nanite) for eve in events]
        evens = [(nanite, node, time, level, mav) for nanite, events in self.VASE_EVENT.items()
        for time, node, level, mav, *_ in events]
        nanite, node, time, level, mav = zip(*evens)
        # level = [8*(8-lev) for lev in level]
        nanite = [300-lev for lev in nanite]
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        #
        # Create an X-Y mesh of the same dimension as the 2D data. You can
        # think of this as the floor of the plot.
        #
        # x_data, y_data = np.meshgrid( np.arange(data_array.shape[1]),
        #                               np.arange(data_array.shape[0]) )
        #
        # Flatten out the arrays so that they may be passed to "ax.bar3d".
        # Basically, ax.bar3d expects three one-dimensional arrays:
        # x_data, y_data, z_data. The following call boils down to picking
        # one entry from each array and plotting a bar to from
        # (x_data[i], y_data[i], 0) to (x_data[i], y_data[i], z_data[i]).
        #
        x_data = time
        y_data = nanite
        z_data = data_array.flatten()
        ax.bar3d( x_data,
                  y_data,
                  np.zeros(len(z_data)),
                  1, 1, z_data )
        #
        # Finally, display the plot.
        #
        plt.show()
        
    def h3dplot(self):
        import numpy as np
        # from mpl_toolkits.mplot3d import Axes3D
        # import matplotlib.pyplot as plt
        from matplotlib import cm
        evens = [(nanite, node, time//10, level, mav) for nanite, events in self.VASE_EVENT.items()
                for time, node, level, mav, *_ in events]
        ''''''
        event = {}
        for nanite, events in self.VASE_EVENT.items():
            [event.setdefault(eve[0], []).append(nanite) for eve in events]
        nanite, node, time, level, mav = zip(*evens)
        # level = [8*(8-lev) for lev in level]
        time = [int(tm)//10 for tm in time]
        # nanite = [300-lev for lev in nanite]
        result=[[0]*len(time)]*len(nanite)

        for nanite, event in self.VASE_EVENT.items():
            for time, node, level, mav, *_ in event:
                #print(nanite, time, node, level, mav)
                result[nanite][int(time)] = level
                # [print(lv[:30]) for lv in result[:30]]

        result = np.array(result, dtype=np.int)
        # return
        fig=plt.figure(figsize=(5, 5), dpi=150)
        ax1=fig.add_subplot(111, projection='3d')

        xlabels = np.array(time)
        xpos = np.arange(xlabels.shape[0])
        ylabels = np.array(nanite)
        ypos = np.arange(ylabels.shape[0])

        xposM, yposM = np.meshgrid(xpos, ypos, copy=False)

        zpos=result
        zpos = zpos.ravel()

        dx=0.5
        dy=0.5
        dz=zpos

        #ax1.w_xaxis.set_ticks(xpos + dx/2.)
        #ax1.w_xaxis.set_ticklabels(xlabels)

        #ax1.w_yaxis.set_ticks(ypos + dy/2.)
        #ax1.w_yaxis.set_ticklabels(ylabels)

        values = np.linspace(0.2, 1., xposM.ravel().shape[0])
        colors = cm.rainbow(values)
        ax1.bar3d(xposM.ravel(), yposM.ravel(), dz*0, dx, dy, dz, color=colors)
        plt.show()

    def dframe(self):
        %matplotlib notebook
        # from matplotlib import cm
        from pandas import DataFrame as DtF
        fields = "nanite, node, time, level, mav".split(", ")
        event = {}
        for nanite, events in self.VASE_EVENT.items():
            [event.setdefault(eve[0], []).append(nanite) for eve in events]
        evens = [(nanite, node, time/20, level, mav) for nanite, events in self.VASE_EVENT.items()
        for time, node, level, mav, *_ in events]
        event_list = zip(*evens)
        df = DtF({campo: valores for campo, valores in zip(fields, event_list) })
        df["emb"] = ["no" if not mav else "lo" if mav <3 else "hi" for mav in df.mav]
        #df.plot.hist(bins=24, alpha=0.5)
        # sns.histplot(data=df,x=df.time)
        #sns.barplot(data=df, x=df.mav, y=df.node) #, col=df.emb)
        # sns.histplot(data=df, x=df.node, bins=20, hue=df.level, alpha=0.5) #, col=df.emb)
        sns.violinplot(data=df, x="mav", y="nanite", linewidth=1).set(title='Distribuição dos Nanites')
        plt.show()
        # sns.histplot(data=df, x=df.nanite, bins=40, hue=df.level, alpha=0.5) #, col=df.emb)
        # sns.catplot(data=df, x=df.emb, y=df.node,hue=df.level, kind="swarm")
        sns.relplot(x="nanite", y="node", hue="level", size="mav",
            sizes=(40, 400), alpha=.5, palette="muted",
            height=6, data=df).set(title='Distribuição dos Nanites pelos Nós')
        '''g = sns.JointGrid(data=df, x="nanite", y="node", space=0)
        g.plot_joint(sns.kdeplot,
                     fill=True, clip=((0, 50), (20, 250)),
                     thresh=0, levels=100, cmap="rocket")
        g.plot_marginals(sns.histplot, color="#03051A", alpha=1, bins=25)'''

        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')

        ax.scatter(df.node, df.nanite, df.mav, color="navy", alpha=0.5,marker="^")
        ax.scatter(df.node, df.nanite, df.level, color="peru", alpha=0.5,marker=".")
        ax.scatter(df.node, df.nanite, df.time, color="green", alpha=0.5,marker="+")
        ax.set_xlabel("Node Id")
        ax.set_ylabel("Nanite Id")
        ax.set_zlabel("Mav/Level/Time")
        ax.set_title("Distribuição dos Nanites")
        plt.show()
        # ++++++++++++++++++++++++++++++++++++++++++++++++++++
        '''
        surf = ax.plot_surface(df.node, df.nanite, df.mav, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

        # Customize the z axis.
        ax.set_zlim(-1, 10)
        # ax.zaxis.set_major_locator(LinearLocator(10))
        # A StrMethodFormatter is used automatically
        ax.zaxis.set_major_formatter('{x:.02f}')

        # Add a color bar which maps values to colors.
        fig.colorbar(surf, shrink=0.5, aspect=5)

        plt.show()
        '''
        return df

        
def main():
    env = simpy.Environment()
    embolization = Embolization(env=env, build_mav=False)
    embolization.main(simulation_time=4)
    report = ReportResults(Embolization.VASE_EVENT)
    report.splot()
    return report.dframe()
    # report.h3dplot()


main()


Running simulation... 
The average wait time is 0 minutes and 4 seconds. 
The count of embolizing nanites is 23 over 83 total,  27.71% efficient. 
The count of embolized vazes is 17 against exisiting 277  mav nodes. 
The process efficacy is  6.14% and over_embolization  1.35 average.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,nanite,node,time,level,mav,emb
0,0,48,0.00,3,3,hi
1,0,39,0.05,4,2,lo
2,0,4,0.10,5,1,lo
3,0,3,0.15,6,0,no
4,0,2,0.20,7,0,no
...,...,...,...,...,...,...
505,80,4,19.55,5,1,lo
506,80,3,19.60,6,0,no
507,80,2,19.65,7,0,no
508,80,1,19.70,7,0,no
